# Formula 1 laptime predictor

This project is an attempt to implement a LSTM neural network to predict Formula 1 laptimes based on previous laptime, circuit, driver ID and the current lap. This is essentially a prototype for a similar network I plan on building in the future which predicts laptimes based on data such as tyre wear, fuel levels etc from the F1 game UDP telemetry.

Dataset: http://ergast.com/mrd/

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt